In [4]:
from tensorflow.python.keras.layers import Layer
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
K = tf.keras.backend

In [21]:
x = tf.constant([[1,2,3],[2,3,4]])
y = tf.reduce_sum(x)
with tf.Session():
  # We can also use 'c.eval()' here.
  print(y.eval())
    
    
    
w= Layer.__dict__.keys()

from deepctr.inputs import DEFAULT_GROUP_NAME
from deepctr.inputs import  build_input_features
from deepctr.inputs import input_from_feature_columns, get_linear_logit, build_input_features, combined_dnn_input, DEFAULT_GROUP_NAME
fm_group=[DEFAULT_GROUP_NAME]
dnn_hidden_units=(128, 128)
l2_reg_linear=0.00001
l2_reg_embedding=0.00001
l2_reg_dnn=0
init_std=0.0001
seed=1024
dnn_dropout=0,
dnn_activation='relu'
dnn_use_bn=False
task='binary'


features = build_input_features(
    linear_feature_columns + dnn_feature_columns)
inputs_list = list(features.values())

from deepctr.inputs import  input_from_feature_columns
group_embedding_dict, dense_value_list = input_from_feature_columns(features, dnn_feature_columns, l2_reg_embedding,
                                                                    init_std, seed, support_group=True)
# group_embedding_dict default_group sparse_emd  (?,1,1)
# dense_value_list dense (?,1)


from deepctr.inputs import input_from_feature_columns, get_linear_logit, build_input_features, combined_dnn_input, DEFAULT_GROUP_NAME
linear_logit = get_linear_logit(features, linear_feature_columns, init_std=init_std, seed=seed, prefix='linear',
                                l2_reg=l2_reg_linear)

15


In [73]:
linear_logit

<tf.Tensor 'no_mask_9_1/no_mask_9/Identity:0' shape=(?, 1) dtype=float32>

In [72]:
features

OrderedDict([('C1', <tf.Tensor 'C1_5:0' shape=(?, 1) dtype=int32>),
             ('C2', <tf.Tensor 'C2_5:0' shape=(?, 1) dtype=int32>),
             ('C3', <tf.Tensor 'C3_5:0' shape=(?, 1) dtype=int32>),
             ('C4', <tf.Tensor 'C4_5:0' shape=(?, 1) dtype=int32>),
             ('C5', <tf.Tensor 'C5_5:0' shape=(?, 1) dtype=int32>),
             ('C6', <tf.Tensor 'C6_5:0' shape=(?, 1) dtype=int32>),
             ('C7', <tf.Tensor 'C7_5:0' shape=(?, 1) dtype=int32>),
             ('C8', <tf.Tensor 'C8_5:0' shape=(?, 1) dtype=int32>),
             ('C9', <tf.Tensor 'C9_5:0' shape=(?, 1) dtype=int32>),
             ('C10', <tf.Tensor 'C10_5:0' shape=(?, 1) dtype=int32>),
             ('C11', <tf.Tensor 'C11_5:0' shape=(?, 1) dtype=int32>),
             ('C12', <tf.Tensor 'C12_5:0' shape=(?, 1) dtype=int32>),
             ('C13', <tf.Tensor 'C13_5:0' shape=(?, 1) dtype=int32>),
             ('C14', <tf.Tensor 'C14_5:0' shape=(?, 1) dtype=int32>),
             ('C15', <tf.Tensor 'C15_5

In [29]:
dnn_feature_columns

[SparseFeat(name='C1', vocabulary_size=526, embedding_dim=1, use_hash=False, dtype='int32', embedding_name='C1', group_name='default_group'),
 SparseFeat(name='C2', vocabulary_size=516, embedding_dim=1, use_hash=False, dtype='int32', embedding_name='C2', group_name='default_group'),
 SparseFeat(name='C3', vocabulary_size=46456, embedding_dim=1, use_hash=False, dtype='int32', embedding_name='C3', group_name='default_group'),
 SparseFeat(name='C4', vocabulary_size=25777, embedding_dim=1, use_hash=False, dtype='int32', embedding_name='C4', group_name='default_group'),
 SparseFeat(name='C5', vocabulary_size=145, embedding_dim=1, use_hash=False, dtype='int32', embedding_name='C5', group_name='default_group'),
 SparseFeat(name='C6', vocabulary_size=12, embedding_dim=1, use_hash=False, dtype='int32', embedding_name='C6', group_name='default_group'),
 SparseFeat(name='C7', vocabulary_size=8057, embedding_dim=1, use_hash=False, dtype='int32', embedding_name='C7', group_name='default_group'),
 S

In [26]:
import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import DeepFM
from deepctr.inputs import  SparseFeat, DenseFeat, get_feature_names


data = pd.read_csv('data/train.csv')

sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']

# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=4)
                       for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                      for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 3.generate input data for model

train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name] for name in feature_names}
test_model_input = {name:test[name] for name in feature_names}

# 4.Define Model,train,predict and evaluate
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy",
              metrics=['binary_crossentropy'], )

history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
pred_ans = model.predict(test_model_input, batch_size=256)
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 64000 samples, validate on 16000 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
64000/64000 - 24s - loss: 0.5181 - binary_crossentropy: 0.5175 - val_loss: 0.5039 - val_binary_crossentropy: 0.5025
Epoch 2/10
64000/64000 - 20s - loss: 0.3732 - binary_crossentropy: 0.3703 - val_loss: 0.5685 - val_binary_crossentropy: 0.5639
Epoch 3/10
64000/64000 - 23s - loss: 0.2429 - binary_crossentropy: 0.2378 - val_loss: 0.6332 - val_binary_crossentropy: 0.6274
Epoch 4/10
64000/64000 - 24s - loss: 0.1913 - binary_crossentropy: 0.1853 - val_loss: 0.7347 - val_binary_crossentropy: 0.7285
Epoch 5/10
64000/64000 - 25s - loss: 0.

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


In [ ]:
features = build_input_features(
    linear_feature_columns + dnn_feature_columns)

inputs_list = list(features.values())

group_embedding_dict, dense_value_list = input_from_feature_columns(features, dnn_feature_columns, l2_reg_embedding,
                                                                    init_std, seed, support_group=True)

linear_logit = get_linear_logit(features, linear_feature_columns, init_std=init_std, seed=seed, prefix='linear',
                                l2_reg=l2_reg_linear)
fm_logit = add_func([FM()(concat_func(v, axis=1))
                     for k, v in group_embedding_dict.items() if k in fm_group])

dnn_input = combined_dnn_input(list(chain.from_iterable(
    group_embedding_dict.values())), dense_value_list)
dnn_output = DNN(dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout,
                 dnn_use_bn, seed)(dnn_input)
dnn_logit = tf.keras.layers.Dense(
    1, use_bias=False, activation=None)(dnn_output)

final_logit = add_func([linear_logit, fm_logit, dnn_logit])

output = PredictionLayer(task)(final_logit)
model = tf.keras.models.Model(inputs=inputs_list, outputs=output)


In [82]:
[v for k,v in group_embedding_dict.items() if k in fm_group]

[[<tf.Tensor 'sparse_emb_C1_1/embedding_lookup/Identity_1:0' shape=(?, 1, 1) dtype=float32>,
  <tf.Tensor 'sparse_emb_C2_1/embedding_lookup/Identity_1:0' shape=(?, 1, 1) dtype=float32>,
  <tf.Tensor 'sparse_emb_C3_1/embedding_lookup/Identity_1:0' shape=(?, 1, 1) dtype=float32>,
  <tf.Tensor 'sparse_emb_C4_1/embedding_lookup/Identity_1:0' shape=(?, 1, 1) dtype=float32>,
  <tf.Tensor 'sparse_emb_C5_1/embedding_lookup/Identity_1:0' shape=(?, 1, 1) dtype=float32>,
  <tf.Tensor 'sparse_emb_C6_1/embedding_lookup/Identity_1:0' shape=(?, 1, 1) dtype=float32>,
  <tf.Tensor 'sparse_emb_C7_1/embedding_lookup/Identity_1:0' shape=(?, 1, 1) dtype=float32>,
  <tf.Tensor 'sparse_emb_C8_1/embedding_lookup/Identity_1:0' shape=(?, 1, 1) dtype=float32>,
  <tf.Tensor 'sparse_emb_C9_1/embedding_lookup/Identity_1:0' shape=(?, 1, 1) dtype=float32>,
  <tf.Tensor 'sparse_emb_C10_1/embedding_lookup/Identity_1:0' shape=(?, 1, 1) dtype=float32>,
  <tf.Tensor 'sparse_emb_C11_1/embedding_lookup/Identity_1:0' shape=(

In [15]:
class FM(Layer):
    """Factorization Machine models pairwise (order-2) feature interactions
     without linear term and bias.

      Input shape
        - 3D tensor with shape: ``(batch_size,field_size,embedding_size)``.

      Output shape
        - 2D tensor with shape: ``(batch_size, 1)``.

      References
        - [Factorization Machines](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf)
    """

    def __init__(self, **kwargs):

        super(FM, self).__init__(**kwargs)

    def build(self, input_shape):
        if len(input_shape) != 3:
            raise ValueError("Unexpected inputs dimensions % d,\
                             expect to be 3 dimensions" % (len(input_shape)))

        super(FM, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, inputs, **kwargs):

        if K.ndim(inputs) != 3:
            raise ValueError(
                "Unexpected inputs dimensions %d, expect to be 3 dimensions"
                % (K.ndim(inputs)))

        concated_embeds_value = inputs

        square_of_sum = tf.square(reduce_sum(
            concated_embeds_value, axis=1, keep_dims=True))
        sum_of_square = reduce_sum(
            concated_embeds_value * concated_embeds_value, axis=1, keep_dims=True)
        cross_term = square_of_sum - sum_of_square
        cross_term = 0.5 * reduce_sum(cross_term, axis=2, keep_dims=False)

        return cross_term

    def compute_output_shape(self, input_shape):
        return (None, 1)